# **🏠 부동산 실거래가 Data Setting**
> 실거래가 예측을 위해 필요한 데이터를 정리해두고 각각 만들어두기
> 집값에 영향을 미칠것 같은 요인들
> - 역세권
> - 유동인구가 많은 강남권
> - 한강뷰
> - 전용면적에 따른 방, 화장실 개수
> - 아파트의 재개발 여부
> - 금리와 GDP
> - 교육 학군
> - 주변 조경 및 시설(숲, 공원, 대형마트, 코엑스, ...)
> 
> 이중 역세권, 유동인구가 많은 지역, 한강뷰, 전용면적에 따른 방, 화장실 개수, 아파트의 재개발 여부를 합치고자 한다.




## Contents
- Library Import
- Data Load
- Crawling of Zigbang home page


## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

## 2. Data Load

In [2]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
add = pd.read_csv('../Data/address_to_latlon_complete.csv')
train = pd.read_csv('../Data/train_latlon_complete.csv')

### 2-1. Setting

- 단순히 아파트명으로 검색하면 현대 아파트는 여러개가 나오기 때문에 동 + 아파트명으로 위치를 분리해주자

In [5]:
# 시, 구, 동으로 분리
train[['시', '구', '동']] = train['시군구'].str.split(expand = True)

In [6]:
# 동 + 아파트명으로 검색어를 구체화해서 동아파트명 변수를 생성
dap = train[['동', '아파트명']].drop_duplicates(['아파트명']).reset_index()

In [7]:
# 동 + 아파트명으로 검색어를 구체화해서 동아파트명 변수를 생성
dap['동아파트명'] = dap['동'] + ' ' + dap['아파트명']

In [87]:
# 확인
dap[dap['동아파트명'] == '한강로2가 신용산빌딩']

,index,동,아파트명,동아파트명
6538,1113986,한강로2가,신용산빌딩,한강로2가 신용산빌딩


### 2-2. crawling

- 방의 개수, 화장실 개수 등 추가 데이터를 얻기 위해 크롤링을 진행

In [9]:
ChromeDriverManager().install()
browser = webdriver.Chrome() # 우리가 컨트롤 할 수 있는 브라우저가 실행이된다.

In [43]:
# 열 이름을 리스트로 지정
column_names = ['동아파트명', '정보1', '정보2']

# 빈 데이터프레임 생성
apt_info = pd.DataFrame(columns=column_names)

In [90]:
for aptment in dap['동아파트명'][6539:6540]:
    print(f"실행중인 아파트 : {aptment}")
    # 브라우저
    browser.get('https://www.zigbang.com/')

    # 검색어 입력
    browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/div/div[2]/div[1]/input').send_keys(aptment)

    time.sleep(1)

    # 검색
    browser.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div[1]/div[2]/div/div[2]/div[2]').click()
    
    time.sleep(1.5)

    # censored
    c = 1

    try:
        i = 3
        while True:

            # 면적
            browser.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div[17]/div/div/div/div/div[1]/div/div/div[{i}]').click()
            
            time.sleep(0.4)

            c = 0

            # 정보1
            info1 = browser.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div[21]/div/div[1]/div[2]/div[2]').text            
            
            # 정보2
            info2 = browser.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/div/div/div[1]/div[1]/div/div[21]/div/div[1]/div[2]/div[3]').text
            
            # 새로운 행 데이터를 딕셔너리로 생성
            new_row_data = {'동아파트명': aptment, '정보1': info1, '정보2': info2}

            # DataFrame에 새로운 행 추가
            apt_info = apt_info.append(new_row_data, ignore_index=True)

            i += 1
            
    except:
        if c == 1:
           # 새로운 행 데이터를 딕셔너리로 생성
            new_row_data = {'동아파트명': aptment, '정보1': 'NotSearched', '정보2': 'NotSearched'}

            # DataFrame에 새로운 행 추가
            apt_info = apt_info.append(new_row_data, ignore_index=True)

In [89]:
# 크롤링 결과 확인
apt_info

,동아파트명,정보1,정보2
0,개포동 개포6차우성,NotSearched,NotSearched
1,개포동 개포우성3차,NotSearched,NotSearched
2,개포동 개포자이,공급 160.71㎡ / 전용 134.3㎡,방 4개 · 욕실 2개 · 계단식
3,개포동 개포자이,공급 183.14㎡ / 전용 153.8㎡,방 4개 · 욕실 2개 · 계단식
4,개포동 개포자이,공급 201.74㎡ / 전용 170.32㎡,방 4개 · 욕실 2개 · 계단식
...,...,...,...
15535,옥인동 옥인시민,NotSearched,NotSearched
15536,역삼동 진달래3차,NotSearched,NotSearched
15537,구로동 807-39,NotSearched,NotSearched
15538,석촌동 밀란체,NotSearched,NotSearched


In [86]:
# 결과 저장
apt_info.to_csv('../Data/temp_roll.csv', index = False)